In [1]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local").setAppName("spark_sql_basic1")
sc   = SparkContext(conf=conf)

24/12/10 13:29:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
movies_rdd = sc.parallelize([
    (1, ("어벤져스", "마블")),
    (2, ("슈퍼맨", "DC")),
    (3, ("배트맨", "DC")),
    (4, ("겨울왕국", "디즈니")),
    (5, ("아이언맨", "마블"))
])


attendances_rdd = sc.parallelize([
    (1, (13934592, "KR")),
    (2, (2182227,"KR")),
    (3, (4226242, "KR")),
    (4, (10303058, "KR")),
    (5, (4300365, "KR"))
])

# 관객수가 500만 이상인 영화를 가져오기
1. 조인 > 필터 > 영화정보
2. 필터 > 조인 > 영화정보

In [4]:
# 1번 방법
movies_att = movies_rdd.join(attendances_rdd)
movies_att.filter(
    lambda x:x[1][1][0] > 5000000
).collect()

[(4, (('겨울왕국', '디즈니'), (10303058, 'KR'))),
 (1, (('어벤져스', '마블'), (13934592, 'KR')))]

In [5]:
# 2번 방법
filtered_att = attendances_rdd.filter(lambda x:x[1][0]>5000000)
movies_rdd.join(filtered_att).collect()

[(4, (('겨울왕국', '디즈니'), (10303058, 'KR'))),
 (1, (('어벤져스', '마블'), (13934592, 'KR')))]

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("spark_sql_basic").getOrCreate()

In [6]:
movies = [
    (1, "어벤져스", "마블", 2012, 4, 26),
    (2, "슈퍼맨", "DC", 2013, 6, 13),
    (3, "배트맨", "DC", 2008, 8, 6),
    (4, "겨울왕국", "디즈니", 2014, 1, 16),
    (5, "아이언맨", "마블", 2008, 4, 30)
]

In [7]:
movie_schema = ['id', 'name', 'company', 'year', 'month', 'day']

# 1. DataFrame 만들기

In [8]:
df = spark.createDataFrame(data=movies, schema= movie_schema)

In [9]:
df.dtypes

[('id', 'bigint'),
 ('name', 'string'),
 ('company', 'string'),
 ('year', 'bigint'),
 ('month', 'bigint'),
 ('day', 'bigint')]

In [10]:
df.show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  3|  배트맨|     DC|2008|    8|  6|
|  4|겨울왕국| 디즈니|2014|    1| 16|
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+



In [11]:
df.select('name').show()

+--------+
|    name|
+--------+
|어벤져스|
|  슈퍼맨|
|  배트맨|
|겨울왕국|
|아이언맨|
+--------+



In [12]:
df.filter(df.year>=2000).show() #col 객체

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  3|  배트맨|     DC|2008|    8|  6|
|  4|겨울왕국| 디즈니|2014|    1| 16|
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+



# SQL을 사용하기 위해 View 등록

In [13]:
df.createOrReplaceTempView('movies')  #view name, table name

In [14]:
#view 를 select projection하는 문장

query = '''
select name from movies
'''

spark.sql(query).show()


+--------+
|    name|
+--------+
|어벤져스|
|  슈퍼맨|
|  배트맨|
|겨울왕국|
|아이언맨|
+--------+



In [15]:
#2010 년 이후 개봉한 영화 조회

query = '''
select name, company
from movies
where year>=2010
'''

In [16]:
spark.sql(query).show()

+--------+-------+
|    name|company|
+--------+-------+
|어벤져스|   마블|
|  슈퍼맨|     DC|
|겨울왕국| 디즈니|
+--------+-------+



In [27]:
# 회사가 마블인 영화 목록
query = '''
select name, company
from movies
where company='마블'
'''

In [28]:
spark.sql(query).show()

+--------+-------+
|    name|company|
+--------+-------+
|어벤져스|   마블|
|아이언맨|   마블|
+--------+-------+



In [ ]:
# 맨으로 끝나는 영화 추출
where name like '%맨'

# ~이~가 들어가는 영화 추출
where name like '%이%'

# 개봉일이 4~8월 사이
where month between 4 and 8

In [ ]:
# and, or

# ~맨으로 끝나면서 개봉연도가 2010년 이하
where name like '%맨' and
year <= 2010

# 회사가 마블이거나 dc인 영화
where company ='마블' OR company = 'dc'

# 선생님 방법
where company in ('마블', 'DC')

# 회사가 "마"로 시작을 하거나, "니"로 끝나는 영화
where company like '마%' or company like '%니'

# 회사가 "마"로 시작을 하거나, "니"로 끝나는 영화 중 2010년 이후로 개봉한 영화
where (company like '마%' or company like '%니') 
and year >= 2010

In [ ]:
# 개봉 연도 오름차순으로 확인
order by year ASC  #내림차순 : DESC

In [ ]:
# count(*) : count는 특정 조건을 만족하는 행의 수를 셈. Null값 포함
# count(name) : 특정 열의 Null이 아닌 값의 수를 셈.
# mean : AVG() 특정 열의 평균값 계산
# sum : 특정 열의 모든 값 합산

In [30]:
attendances = [
    (1, 13934592., "KR"),
    (2, 2182227.,"KR"),
    (3, 4226242., "KR"),
    (4, 10303058., "KR"),
    (5, 4300365., "KR")
]

In [31]:
from pyspark.sql.types import StringType, IntegerType, FloatType, StructType, StructField

att_schema = StructType([ # 모든 컬럼의 타입을 통칭 - 컬럼 데이터의 집합
    StructField("id", IntegerType(), True), # StructField : 컬럼
    StructField("att", FloatType(), True),
    StructField("theater_country", StringType(), True)
])

In [32]:
att_df = spark.createDataFrame( data=attendances, schema = att_schema )
att_df.dtypes

[('id', 'int'), ('att', 'float'), ('theater_country', 'string')]

In [33]:
att_df.show()

+---+-----------+---------------+
| id|        att|theater_country|
+---+-----------+---------------+
|  1|1.3934592E7|             KR|
|  2|  2182227.0|             KR|
|  3|  4226242.0|             KR|
|  4|1.0303058E7|             KR|
|  5|  4300365.0|             KR|
+---+-----------+---------------+



In [34]:
att_df.createOrReplaceTempView('att')

In [35]:
att_df.select('*').show()

+---+-----------+---------------+
| id|        att|theater_country|
+---+-----------+---------------+
|  1|1.3934592E7|             KR|
|  2|  2182227.0|             KR|
|  3|  4226242.0|             KR|
|  4|1.0303058E7|             KR|
|  5|  4300365.0|             KR|
+---+-----------+---------------+



In [36]:
query = '''
select *
from movies JOIN att ON movies.id = att.id
'''

In [37]:
spark.sql(query).show()

+---+--------+-------+----+-----+---+---+-----------+---------------+
| id|    name|company|year|month|day| id|        att|theater_country|
+---+--------+-------+----+-----+---+---+-----------+---------------+
|  5|아이언맨|   마블|2008|    4| 30|  5|  4300365.0|             KR|
|  1|어벤져스|   마블|2012|    4| 26|  1|1.3934592E7|             KR|
|  3|  배트맨|     DC|2008|    8|  6|  3|  4226242.0|             KR|
|  2|  슈퍼맨|     DC|2013|    6| 13|  2|  2182227.0|             KR|
|  4|겨울왕국| 디즈니|2014|    1| 16|  4|1.0303058E7|             KR|
+---+--------+-------+----+-----+---+---+-----------+---------------+



In [38]:
# 중복된 열이 싫다면

query = '''
SELECT 
    movies.id, 
    name, 
    company, 
    year, 
    month, 
    day, 
    att, 
    theater_country
FROM 
    movies 
JOIN 
    att 
ON 
    movies.id = att.id

'''

In [39]:
spark.sql(query).show()

+---+--------+-------+----+-----+---+-----------+---------------+
| id|    name|company|year|month|day|        att|theater_country|
+---+--------+-------+----+-----+---+-----------+---------------+
|  5|아이언맨|   마블|2008|    4| 30|  4300365.0|             KR|
|  1|어벤져스|   마블|2012|    4| 26|1.3934592E7|             KR|
|  3|  배트맨|     DC|2008|    8|  6|  4226242.0|             KR|
|  2|  슈퍼맨|     DC|2013|    6| 13|  2182227.0|             KR|
|  4|겨울왕국| 디즈니|2014|    1| 16|1.0303058E7|             KR|
+---+--------+-------+----+-----+---+-----------+---------------+



In [48]:
#  ➕ 다른 방법 : using. 단 컬럼 이름이 같아야 함. (RENAME후 USING이 나을 듯함.)
query = '''
SELECT *
FROM movies 
JOIN att 
USING (id)
'''

# 컬럼 이름 다를 때 : ALTER TABLE att RENAME COLUMN attendance_id TO id;

In [41]:
spark.sql(query).show()

+---+--------+-------+----+-----+---+-----------+---------------+
| id|    name|company|year|month|day|        att|theater_country|
+---+--------+-------+----+-----+---+-----------+---------------+
|  5|아이언맨|   마블|2008|    4| 30|  4300365.0|             KR|
|  1|어벤져스|   마블|2012|    4| 26|1.3934592E7|             KR|
|  3|  배트맨|     DC|2008|    8|  6|  4226242.0|             KR|
|  2|  슈퍼맨|     DC|2013|    6| 13|  2182227.0|             KR|
|  4|겨울왕국| 디즈니|2014|    1| 16|1.0303058E7|             KR|
+---+--------+-------+----+-----+---+-----------+---------------+



In [42]:
moviews_views = spark.sql(query)

In [43]:
moviews_views.createOrReplaceTempView('movies_view')

In [44]:
query = '''
select * from movies_view
'''

In [45]:
spark.sql(query).show()

+---+--------+-------+----+-----+---+-----------+---------------+
| id|    name|company|year|month|day|        att|theater_country|
+---+--------+-------+----+-----+---+-----------+---------------+
|  5|아이언맨|   마블|2008|    4| 30|  4300365.0|             KR|
|  1|어벤져스|   마블|2012|    4| 26|1.3934592E7|             KR|
|  3|  배트맨|     DC|2008|    8|  6|  4226242.0|             KR|
|  2|  슈퍼맨|     DC|2013|    6| 13|  2182227.0|             KR|
|  4|겨울왕국| 디즈니|2014|    1| 16|1.0303058E7|             KR|
+---+--------+-------+----+-----+---+-----------+---------------+



In [49]:
sc.stop()

In [50]:
spark.stop()